# Transer Learning

## Import all necessary libraries

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models

## Load the CIFAR1-10 Dataset

In [3]:
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:05<00:00, 29293121.93it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Load pre-trained ResNet18 Model

In [4]:
model = models.resnet18(pretrained=True)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 209MB/s]


## Freeze all layers to avoid re-training the pre-trained model

In [5]:
for param in model.parameters():
    param.requires_grad = False

## Replace the last fully connected layer to match the CIFAR-10 classes

In [6]:
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)

## Define the loss function and optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

## Select device

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

## Train the model

In [9]:
for epoch in range(5):
    running_loss = 0.0 
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        print(f"Epoch [{epoch + 1}/5], Loss: {running_loss / len(trainloader):.4f}")
    

Epoch [1/5], Loss: 0.0015
Epoch [1/5], Loss: 0.0031
Epoch [1/5], Loss: 0.0047
Epoch [1/5], Loss: 0.0062
Epoch [1/5], Loss: 0.0076
Epoch [1/5], Loss: 0.0091
Epoch [1/5], Loss: 0.0107
Epoch [1/5], Loss: 0.0122
Epoch [1/5], Loss: 0.0137
Epoch [1/5], Loss: 0.0151
Epoch [1/5], Loss: 0.0165
Epoch [1/5], Loss: 0.0180
Epoch [1/5], Loss: 0.0194
Epoch [1/5], Loss: 0.0208
Epoch [1/5], Loss: 0.0223
Epoch [1/5], Loss: 0.0237
Epoch [1/5], Loss: 0.0250
Epoch [1/5], Loss: 0.0265
Epoch [1/5], Loss: 0.0279
Epoch [1/5], Loss: 0.0293
Epoch [1/5], Loss: 0.0307
Epoch [1/5], Loss: 0.0320
Epoch [1/5], Loss: 0.0334
Epoch [1/5], Loss: 0.0347
Epoch [1/5], Loss: 0.0361
Epoch [1/5], Loss: 0.0374
Epoch [1/5], Loss: 0.0386
Epoch [1/5], Loss: 0.0399
Epoch [1/5], Loss: 0.0412
Epoch [1/5], Loss: 0.0426
Epoch [1/5], Loss: 0.0441
Epoch [1/5], Loss: 0.0453
Epoch [1/5], Loss: 0.0466
Epoch [1/5], Loss: 0.0479
Epoch [1/5], Loss: 0.0491
Epoch [1/5], Loss: 0.0503
Epoch [1/5], Loss: 0.0515
Epoch [1/5], Loss: 0.0527
Epoch [1/5],

## Evaluate model on test data

In [10]:
correct = 0
total = 0
model.eval()

with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        
        correct += (predicted == labels).sum().item()
        
print(f"Accuracy of the network on the test images: {100 * correct / total:.2f}%")

Accuracy of the network on the test images: 80.54%
